In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("/kaggle/input/mnist-in-csv/mnist_train.csv")
data = np.array(data)
np.random.shuffle(data)
data = data.T
m, n = data.shape
y = data[0]
X = data[1:]

In [ ]:
def init_params():
    W1 = np.random.randn(10, 784) * 0.01
    W2 = np.random.randn(10, 10) * 0.01
    B1 = np.random.randn(10, 1) * 0.01
    B2 = np.random.randn(10, 1) * 0.01
    return W1, B1, W2, B2

def ReLu(Z):
    return np.maximum(0, Z)

def Softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / np.sum(expZ, axis=0, keepdims=True)

def forward_prop(X, y, W1, B1, W2, B2):
    Z1 = W1.dot(X) + B1
    A1 = ReLu(Z1)
    Z2 = W2.dot(A1) + B2
    A2 = Softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(_y):
    one_hot_y = np.zeros((_y.size, _y.max() + 1))
    one_hot_y[np.arange(_y.size), _y] = 1
    return one_hot_y.T

def ReLuDerivative(Z):
    return Z > 0

def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    Y = one_hot(Y)
    dZ2 = A2 - Y
    dW2 = (1 / m) * dZ2.dot(A1.T)
    dB2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLuDerivative(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T)
    dB1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, dB1, dW2, dB2

def update_params(W1, B1, W2, B2, dW1, dB1, dW2, dB2, alpha):
    W1 = W1 - alpha * dW1
    W2 = W2 - alpha * dW2
    B1 = B1 - alpha * dB1
    B2 = B2 - alpha * dB2
    return W1, B1, W2, B2


In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(preds, Y):
    return np.sum(preds == Y) / Y.size


def gradient_descent(X, y, epochs=1000, alpha=0.01):
    W1, B1, W2, B2 = init_params()
    for i in range(epochs):
        Z1, A1, Z2, A2 = forward_prop(X, y, W1, B1, W2, B2)
        dW1, dB1, dW2, dB2 = back_prop(Z1, A1, Z2, A2, W2, X, y)
        W1, B1, W2, B2 = update_params(W1, B1, W2, B2, dW1, dB1, dW2, dB2, alpha)
        if i % 100 == 0:
            print(f"Iteration {i}: Accuracy = {get_accuracy(get_predictions(A2), y):.4f}")

    return W1, B1, W2, B2

In [ ]:
W1, B1, W2, B2 = gradient_descent(X, y)